---
# CVL - Get started
This is a Jupyter Notebook to get you started on a typical CVL workflow.

---
# Import libraries
This is how to import libraries (also called packages) which are required later in the code.

In [ ]:
import os
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
import warnings
warnings.filterwarnings("ignore")

---
# Read in data

## A) Working with data products remote (supporting OPeNDAP)
NetCDF data for instance can be imported directly and we can read either the entire dataset or a subset of it. It is possible to filter by spatial and temporal dimensions.

For documentation how to work with xarray library, please visit https://xarray.pydata.org/

`open_dataset` method will open the filehandle but not read the entire contents into memory. `load_dataset` method will load the contents into memory

In [ ]:
# specify dataset url (e.g. Svalbard weather station)
data_url = 'https://thredds.met.no/thredds/dodsC/met.no/observations/stations/SN99840.nc'

# reading datasets:
# decode_times option will allow to manipulate timeseries using time variable

dst = xr.open_dataset(data_url, decode_times=True)

Lets check out what is on the dataset

In [ ]:
dst

We want to extract air temperature and take only a subset of the dataset

In [ ]:
# data subset after 1979
tmp = dst['air_temperature_2m'].where(dst['time.year']>=1979,drop=True)

# filter data
air_temperature_2m = tmp.where(tmp<300)

Create a figure and plot the extracted data

In [ ]:
fig = plt.figure(figsize=(15,5))
air_temperature_2m.plot.line(x='time')

## B) Working with data products locally (not supporting OPeNDAP)
In case data can not be accessed using the OPEnDAP protocol, a local copy of the dataset is required. We will use use `requests` library to obtain the data over HTTP and `tempfile` mmodule to remove the file after the information is extracted.

In [ ]:
import requests
import tempfile

Instead of creating a permament copy of a file it is possible to load it only temporarily using `tempfile` Python module. After the data is read from the file into memory the temporary location will be automatically removed.

In [ ]:
# url to data file
data_url = 'https://api.npolar.no/dataset/5f53146f-0489-4400-a94c-59a6d565bf32/_file/atw12_CTD.nc'

# create temporary directory
with tempfile.TemporaryDirectory() as tmpdirname:

    fpath = os.path.join(tmpdirname, 'tmp.nc')
    
    # create a request link to the file
    r = requests.get(data_url, allow_redirects=True)
    
    # open a NetCDF file and write the linked file into it
    open(fpath, 'wb').write(r.content)
    
    # load the dataset on to a variable
    # it will be available after the temporary directory has been deleted agian
    dst = xr.open_dataset(fpath)

From this point the steps are similar to example **A)** and you can extract a subset of the data and plot it into a figure

In [ ]:
fig = plt.figure(figsize=(15,5))
dst['TEMP'].plot(x='STATION',y='PRES')
plt.gca().invert_yaxis()

---
# Load satellite data

Let's add satellite data to the previous plot. We will use Sentinel-2 scene from the CVL data portal served by OPeNDAP server.

In [ ]:
url = "https://nbstds.met.no/thredds/dodsC/NBS/S2B/2022/09/21/S2B_MSIL1C_20220921T125739_N0400_R038_T33XVJ_20220921T150723.nc"
s2_dst = xr.open_dataset(url)
s2_dst

# Partial loading of the dataset
By specifying how many array elements to skip we can load only Nth element for data preview.

---

In [ ]:
stride = 5
ref = s2_dst["B4"][0,::stride,::stride]
lon = s2_dst['lon'][::stride,::stride]
lat = s2_dst['lat'][::stride,::stride]

---
# Create a map in custom projection

Below an example of how to create a simple map using the cartopy package.

In [ ]:
# define projections for the figure and the original set of coordinates
orig_projection = ccrs.PlateCarree()
target_projection = ccrs.Stereographic(central_longitude=0,central_latitude=60.)

# create a figure handle
fig = plt.figure(figsize=(12,12))

# create an axes handle with the previously specified projection
ax = plt.axes(projection=target_projection)

# customise map and add features
ax.set_extent([10,20,79,80])
plt.pcolormesh(lon, lat, ref, transform=orig_projection)

ax.gridlines(color='white',draw_labels=True, y_inline=False)
ax.coastlines(color='yellow')
ax.add_feature(cartopy.feature.LAND)
ax.add_feature(cartopy.feature.OCEAN)